In [1]:
import dspy

from diversity_gen import OptDiverseDataGenerator
import pandas
from diversity_metrics import dc_score, negative_cosine_sim, cosine_sim
import random
import json

from dotenv import load_dotenv
load_dotenv(".env")


def metric(gold, pred, trace=None):
    computed_dc_score = dc_score(pred.seen_data + pred.generated_data)
    computed_cos_score = cosine_sim(gold.gold_examples, pred.curr_gens)
    if computed_cos_score > 0.6:
        computed_cos_score = 1
    elif computed_cos_score < 0.4:
        computed_cos_score = 1
    computed_neg_cos_sim = negative_cosine_sim(pred.seen_data + pred.generated_data)
    overall_score = computed_dc_score - computed_cos_score + computed_neg_cos_sim
    return overall_score

def metric_separate(gold, pred):
    computed_dc_score = dc_score(pred.seen_data + pred.generated_data)
    computed_cos_score = cosine_sim(gold.gold_examples, pred.curr_gens)
    computed_neg_cos_sim = negative_cosine_sim(pred.seen_data + pred.generated_data)
    return dspy.Prediction(
        diversity_score=computed_dc_score,
        cosine_sim_ref_pred=computed_cos_score,
        diversity_cos_score=computed_neg_cos_sim
    )

def gepa_metric(gold, pred, trace=None, pred_name=None, pred_trace=None):
    metric_score = metric_separate(gold, pred)
    overall_score = metric(gold, pred, trace)
    
    feedback_text = f"The overall score is {overall_score:.2f}, which computed as the cosine similarity between the in-context gold examples and generations ({metric_score.cosine_sim_ref_pred: .2f}) subtracted from the sum of two diversity scores (DC Score = {metric_score.diversity_score: .2f}, Negative Cosine Similarity = {metric_score.diversity_cos_score: .2f}). Try to improve the diversity of your response. The generations should be sufficiently similar to the in-context gold examples without being too similar."
    if metric_score.cosine_sim_ref_pred > 0.6:
        feedback_text += " The current cosine similarity between the in-context gold examples and the generations is too high. Aim to be more creative in the generations while adhering to the hard requirements."
        metric_score.cosine_sim_ref_pred = -10
    elif metric_score.cosine_sim_ref_pred < 0.4:
        feedback_text += " The current cosine similarity between the in-context gold examples and the generations is too low. Adhere to the hard requirements and still have generations to be sufficiently similar to the gold examples."
        metric_score.cosine_sim_ref_pred = -1
    return dspy.Prediction(
        score=overall_score,
        feedback=feedback_text,
    )


/home/siyanli/miniconda3/envs/omni/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


[2025-11-10 11:42:01,353] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/siyanli/miniconda3/envs/omni/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.6
 [WARNING]  using untested triton version (3.2.0), only 1.0.0 is known to be compatible


/home/siyanli/miniconda3/envs/omni/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/siyanli/miniconda3/envs/omni/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


In [2]:
pupa_tnb_data = pandas.read_csv("PUPA_TNB.csv")
random.seed(42)
random_sample = pupa_tnb_data.sample(n=20)

all_examples = []

for i, row in random_sample.iterrows():
    if not pandas.isna(row["user_query"]):
        curr_example = "User Query: " + row["user_query"] + "\nAssistant Response: " + row["target_response"]
        all_examples.append(curr_example)    

lm = dspy.LM("gpt-4.1-nano", cache=True)
dspy.configure(lm=lm)

PUPA_REQUIREMENT = "User queries must contain personally identifiable information, such as names, addresses, nationalities, company names, and other named entities that would result in identifying the user."

task_gen = OptDiverseDataGenerator()

# Start creating actual data for opt
dspy_examples = []

for _ in range(250):
    dspy_examples.append(dspy.Example({"gold_examples": random.choices(all_examples, k=3),
                                        "hard_requirement": PUPA_REQUIREMENT}).with_inputs("gold_examples", "hard_requirement"))
    
train_set = dspy_examples[:200]
dev_set = dspy_examples[200:]
tiny_dev = dspy_examples[240:]


In [3]:

eval = dspy.Evaluate(metric=metric, devset=dev_set, return_all_scores=True)


In [4]:
from dspy import GEPA

gepa = GEPA(metric=gepa_metric, track_stats=True, 
            reflection_lm=dspy.LM(model='gpt-4.1', temperature=1.0, max_tokens=32000),
            track_best_outputs=True, max_full_evals=2)
new_prog = gepa.compile(task_gen, trainset=train_set[:30], valset=dev_set[:15])
pareto_frontier = new_prog.detailed_results.val_aggregate_scores

2025/11/10 11:42:12 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 90 metric calls of the program. This amounts to 2.00 full evals on the train+val set.
2025/11/10 11:42:12 INFO dspy.teleprompt.gepa.gepa: Using 15 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/90 [00:00<?, ?rollouts/s]2025/11/10 11:42:45 INFO dspy.evaluate.evaluate: Average Metric: 11.90856537662391 / 15 (79.4%)
2025/11/10 11:42:45 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.793904358441594
GEPA Optimization:  17%|█▋        | 15/90 [00:33<02:46,  2.22s/rollouts]2025/11/10 11:42:45 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.793904358

Average Metric: 1.80 / 3 (59.9%): 100%|██████████| 3/3 [00:12<00:00,  4.16s/it]

2025/11/10 11:42:58 INFO dspy.evaluate.evaluate: Average Metric: 1.7970584965710128 / 3 (59.9%)
2025/11/10 11:42:58 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Exception during reflection/proposal: No valid predictions found for any module.
2025/11/10 11:42:58 INFO dspy.teleprompt.gepa.gepa: Traceback (most recent call last):
  File "/home/siyanli/miniconda3/envs/omni/lib/python3.10/site-packages/gepa/proposer/reflective_mutation/reflective_mutation.py", line 119, in propose
    reflective_dataset = self.adapter.make_reflective_dataset(curr_prog, eval_curr, predictor_names_to_update)
  File "/home/siyanli/miniconda3/envs/omni/lib/python3.10/site-packages/dspy/teleprompt/gepa/gepa_utils.py", line 288, in make_reflective_dataset
    raise Exception("No valid predictions found for any module.")
Exception: No valid predictions found for any module.

2025/11/10 11:42:58 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Reflective mutation did not propose a new candidate
GEPA Optimization:  20%|


Average Metric: 1.63 / 3 (54.4%): 100%|██████████| 3/3 [00:12<00:00,  4.21s/it]

2025/11/10 11:43:10 INFO dspy.evaluate.evaluate: Average Metric: 1.6327892009169611 / 3 (54.4%)


...

Average Metric: 2.08 / 3 (69.2%): 100%|██████████| 3/3 [00:21<00:00,  7.14s/it]

2025/11/10 11:48:50 INFO dspy.evaluate.evaluate: Average Metric: 2.0751110720935273 / 3 (69.2%)
2025/11/10 11:48:50 INFO dspy.teleprompt.gepa.gepa: Iteration 9: Exception during reflection/proposal: No valid predictions found for any module.
2025/11/10 11:48:50 INFO dspy.teleprompt.gepa.gepa: Traceback (most recent call last):
  File "/home/siyanli/miniconda3/envs/omni/lib/python3.10/site-packages/gepa/proposer/reflective_mutation/reflective_mutation.py", line 119, in propose
    reflective_dataset = self.adapter.make_reflective_dataset(curr_prog, eval_curr, predictor_names_to_update)
  File "/home/siyanli/miniconda3/envs/omni/lib/python3.10/site-packages/dspy/teleprompt/gepa/gepa_utils.py", line 288, in make_reflective_dataset
    raise Exception("No valid predictions found for any module.")
Exception: No valid predictions found for any module.

2025/11/10 11:48:50 INFO dspy.teleprompt.gepa.gepa: Iteration 9: Reflective mutation did not propose a new candidate
GEPA Optimization:  77%|


Average Metric: 2.05 / 3 (68.2%): 100%|██████████| 3/3 [00:15<00:00,  5.31s/it]

2025/11/10 11:49:06 INFO dspy.evaluate.evaluate: Average Metric: 2.0472063027997365 / 3 (68.2%)


2025/11/10 11:49:40 INFO dspy.teleprompt.gepa.gepa: Iteration 10: Proposed new text for proposer.predict: You are given a task to generate new user query and assistant response pairs (called "instances") using a set of in-context examples and requirements. The goal is to create additional instances that match the thematic, stylistic, and content-related patterns found in the examples while satisfying explicit constraints described in the requirements.

Input Format:
- You receive inputs containing:
  - **examples**: A list of existing user query and assistant response pairs. Each user query involves a user-specific request that includes identifiable information such as names, company names, addresses, nationalities, or other named entities. The assistant response addresses the user query in a polite, formal, and informative manner (typically as professional correspondence, information requests, technical or process explanations, translations, or interactive tests).
  - **data_summary**

In [20]:
# optimizer = dspy.SIMBA(metric=metric, max_steps=3)
# optimized_program = optimizer.compile(task_gen, trainset=train_set)

# # Save optimize program for future use
# optimized_program.save(f"optimized.json")

In [7]:
new_prog.generated_data

[]

In [16]:
import json

json.dump(optimized_program.generated_data, open("gen_data.json", "w+"))